In [1]:
%aiida demo

Loaded AiiDA DB environment - profile name: demo.

In [2]:
from pymatgen import get_structure_from_mp
from aiida.orm import StructureData
from aiida_user_addons.common.inputset.vaspsets import VASPInputSet
from aiida_user_addons.tools.dryrun import dryrun_vasp, get_jobscheme
from aiida_user_addons.vworkflows.relax import VaspRelaxWorkChain, RelaxOptions

Get the SrTiO3 structure from pymatgen

In [3]:
sto = StructureData(pymatgen=get_structure_from_mp("mp-5229"))

Setup the input for a `VaspCalculation`. 

Note: normally it is more common to use `VaspWorkChain` or `VaspRelaxWorkChain` for running the calculations.

In [4]:
inputset = VASPInputSet("UCLRelaxSet", sto, overrides={'encut': 300})
potential = DataFactory('vasp.potcar').get_potcars_from_structure(
                structure=sto,
                family_name='PBE.54',
                mapping=inputset.get_pp_mapping())


builder = WorkflowFactory('vaspu.vasp').get_builder()
builder.structure = sto
builder.parameters = inputset.get_input_dict(raw_python=False)
builder.potential_family = Str('PBE.54')
builder.potential_mapping = Dict(dict=inputset.get_pp_mapping())
builder.options = Dict(dict={
    'resources': {'num_machines': 1, 'tot_num_mpiprocs': 4}
})
builder.kpoints = inputset.get_kpoints(0.05)
builder.code = Code.get_from_string("vasp@localhost")
builder.auto_parallel = Dict(dict={'vasp_exe': 'vasp_std', 'cpus_per_node': 4})

In [5]:
from aiida.engine import run, submit

In [6]:
run(builder)

09/27/2020 06:57:59 PM <12562> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [INFO] Broadcasting state change of 34525: state_changed.created.running
09/27/2020 06:57:59 PM <12562> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [INFO] Broadcasting state change of 34525: state_changed.running.running
09/27/2020 06:57:59 PM <12562> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [INFO] Broadcasting state change of 34525: state_changed.running.running
09/27/2020 06:57:59 PM <12562> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [INFO] Broadcasting state change of 34525: state_changed.running.running
09/27/2020 06:58:00 PM <12562> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [34525|VaspWorkChain|perform_autoparallel]: Performing local dryrun for auto-parallelisation
09/27/2020 06:58:00 PM <12562> aiida.execmanager: [WARNING] CalcJobNode<34527> already has a `remote_folder` output: skipping upload
09/27/2020 06:58:01 PM <1256

{'parallel_settings': <Dict: uuid: 2b677791-7acc-4ecd-873a-a2b476a9eb6a (pk: 34531)>,
 'misc': <Dict: uuid: 93c443f7-ceca-4ab1-bd52-9646e06927c6 (pk: 34534)>,
 'remote_folder': <RemoteData: uuid: 88e4817f-4e1a-4f20-ba93-b7092e4067a5 (pk: 34536)>,
 'retrieved': <FolderData: uuid: 7f1c1495-373c-4630-ab70-5b189fb58f46 (pk: 34535)>}

## Access the results

In [10]:
work = load_node("4c7d3bd4-ede5-44b5-8119-c4be5e98f4a5")

In [11]:
work.outputs.parallel_settings.get_dict()

{'kpar': 4, 'ncore': 1}

In [12]:
work.outputs.misc.get_dict()

{'symmetries': {'point_group': {'static': [None], 'dynamic': [None]},
  'primitive_translations': [1],
  'num_space_group_operations': {'static': [48], 'dynamic': [48]}},
 'stdout_error': [{'message': 'WARNING: Linux kernel CMA support was requested via the',
   'critical': False,
   'shortname': 'general'}],
 'magnetization': [-0.0001519],
 'maximum_force': 0.0,
 'maximum_stress': 390.40519455,
 'total_energies': {'energy_no_entropy': -39.29650258}}